## Importation des fichiers utiles

In [48]:
import urllib.request
import json
import pandas as pd
import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.linear_model import Perceptron

NOMBRE_IMAGE=141

<br>

## Demande de Json
Nous Faisons une Requète WikiData pour demander un Json contenant le lien menant à des libres d'une certaine catégorie. Nous avons essayé plusieurs types d'images différentes :

mythical creature = Q2239243<br>
photograph = Q125191<br>
flower = Q506<br>
fruit = Q1364<br>
Chien = Q144<br>
lac  = Q23397<br>


SELECT ?image ?im WHERE {
  ?image wdt:P31 wd:Q506.
  ?image wdt:P18 ?im
  }
LIMIT 200

In [49]:
lstSources = {"Fruit":1364,"Dog":144,"Lake":23397}
limitPerSources = 5
sources = {}
tagTemp = []
for key,value in lstSources.items():    #Get all images to download
    url ="https://query.wikidata.org/sparql?query=SELECT%20%3Fimage%20%3Fim%20WHERE%20%7B%0A%20%20%3Fimage%20wdt%3AP31%20wd%3AQ"+str(value)+".%0A%20%20%3Fimage%20wdt%3AP18%20%3Fim%0A%20%20%7D%0ALIMIT%20"+str(limitPerSources)+"&format=json"
    response = urllib.request.urlopen(url)
    myJson = json.loads(response.read().decode("utf-8"))
    
    for image in myJson["results"]["bindings"]:  #Add the tag to the image
        image['tag'] = key
        
    if sources == {}:   #Make one big json containing all images
        sources = myJson
    else:
        sources["results"]["bindings"] = sources["results"]["bindings"]+myJson["results"]["bindings"]

myJsonNormalized = pd.json_normalize(sources)

<br>

## Télechargement des images ##
A partir du Json contenant les liens menant aux images, nous lancons des requètes URLLIB pour télecharger ces images.

In [50]:
#telechargement des images

responseJson = myJsonNormalized["results.bindings"]
index=0
lstTag=[]
for i in range(len(responseJson[0])):   
    imageLink = responseJson[0][i]["im"]["value"]
    lstTag.append(responseJson[0][i]["tag"])
    print(index)
    
    im = "./images/image"+str(index)+".jpg"
    urllib.request.urlretrieve(imageLink, im)
    index +=1

    try:
        imgfile = Image.open(im).convert("RGB")
    except:
        index-=1
        del lstTag[-1]
        print("pas fonctionne")
    
NOMBRE_IMAGE=index

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


<br>

## Récupération des métadonnées
L'obhectif de cette partie est de récupérer les informations des images, nous récupérons les images suivantes :<br>
1. Récupération Exif utiles
2. Récupération couleurs dominantes (K-means)
3. Récupération taille des images
4. Détermination portrait ou paysage

Apres cela, nous avons creer un Json **metaDataJson.json** contenant toutes les informationns precedentes

In [56]:
# Récupération des métadonnées

metaData ={}
for i in range(NOMBRE_IMAGE):
    im = "./images/image"+str(i)+".jpg"
    try:
        imgfile = Image.open(im).convert("RGB")

        #Recupération Exif utiles
        exif = {}
        exifCritere= ("Model","Make","ImageDescription","ExposureTime","GPSInfo","ISO","DateTimeOriginal")
        for k,v in  imgfile.getexif().items() :
            if k in ExifTags.TAGS :
                if ExifTags.TAGS[k] in exifCritere:
                    exif = {ExifTags.TAGS[k]:v}

        # Recupération couleurs dominantes (K-means)
        numarray = np.array(imgfile.getdata(), np.uint8)
        clusters  = MiniBatchKMeans(n_clusters=3, n_init=2) 
        clusters.fit(numarray)
        L = clusters.cluster_centers_.astype(int).tolist()
        if L[0][0]>L[0][1] and L[0][0]>L[0][2]:
            domColor = "Red"
        elif L[0][1]>L[0][0] and L[0][1]>L[0][2]:
            domColor = "Green"
        elif L[0][2]>L[0][0] and L[0][2]>L[0][1]:
            domColor = "Blue"
        elif L[0][0]==L[0][1]==L[0][2]:
            domColor = "White"
        else:
            domColor = "None"

        # Recupération taille des images
        width = imgfile.width
        height = imgfile.height
        mode = imgfile.mode

        # Détermination portrait ou paysage
        if abs(1-width/height)<0.1 :
            orientation='carre' 
        elif width>height:
            orientation='paysage'
        else: 
            orientation="portrait"

        # Get image size (big, small, ...)
        pixelCount = width * height
 
        if pixelCount >= 1920*1080:
            size = "Grande"
        elif pixelCount >= 1280*720:
            size = "Moyenne"
        elif pixelCount >= 720*480:
            size = "Petite"
        elif pixelCount < 720*480:
            size = "Vignette"
        else:
            size = "WTF"

        metaData["image"+str(i)] = { "width" : width, "height": height, "exif": exif, "mode":mode, "tags":[lstTag[i]], "couleurs":L, "orientation":orientation, "size":size, "dom":domColor}
    except Exception:
        print("l'image "+str(i) +" ne peut pas etre ouverte")
metaData

metaDatajson = json.dumps(metaData)
f = open("metaDataJson.json", "w")
f.write(metaDatajson)
f.close()



In [14]:
def analyse(numUser): 
    jsonUser=json.load(open("jsonUser.json"))[numUser]
    jsonMeta=json.load(open("metaDatajson.json"))
    couleurs=[]
    dataArray=[]
    for image in jsonMeta:
        if image in jsonUser["data"]["image"]:
        #image correspond maintenant aux informations des images testees
            L=[]
            dataImage=[]
            if jsonMeta[image]["tags"]==[]:
                dataImage.append(None)
            else:
                dataImage.append(jsonMeta[image]["tags"])
            dataImage.append(jsonMeta[image]["orientation"])
            dataImage.append(jsonMeta[image]["width"])

            for couleur in jsonMeta[image]["couleurs"]:
                for valeurRBG in couleur:
                    L.append(valeurRBG)
            dataArray.append(dataImage)
            couleurs.append(L)
    print(dataArray)
    numarray=np.array(couleurs)
    like=np.array(jsonUser["data"]["resultat"])
    
    #modele des couleurs
    percep = Perceptron(max_iter=1000)
    percep.fit(numarray, like)
    

    #modele arbre de decision generale
    dataFrame = pd.DataFrame(dataArray, columns=["tags", "orientation", "mode"])
    likeFrame=pd.DataFrame(like,columns=['like'])


    le1 = LabelEncoder()
    dataFrame['tags'] = le1.fit_transform(dataFrame['tags'])

    le2 = LabelEncoder()
    dataFrame['orientation'] = le2.fit_transform(dataFrame['orientation'])

    le3 = LabelEncoder()
    dataFrame['mode'] = le3.fit_transform(dataFrame['mode'])

    le4 = LabelEncoder()
    likeFrame['like'] = le4.fit_transform(likeFrame['like'])


[[None, 'portrait', 440], [None, 'paysage', 4509], [None, 'paysage', 640], [None, 'paysage', 407], [None, 'paysage', 4288], [None, 'paysage', 4792], [None, 'paysage', 4792], [None, 'portrait', 425], [None, 'paysage', 2592], [None, 'portrait', 697]]


ValueError: Found input variables with inconsistent numbers of samples: [10, 4]

## Analyse

In [ ]:
def analyse():
      jsonMeta=json.load(open("metaDatajson.json"))
      jsonMetaNormalized = pd.json_normalize(jsonMeta)
      print(jsonMetaNormalized)

      jsonUser=json.load(open("jsonUser.json"))
      jsonUserNormalized = pd.json_normalize(jsonUser)
      
      L=[]
      for i,obj in jsonUserNormalized.iterrows():
            L.append(obj["data.like"])
      print(L)
      numarray = np.array(jsonMetaNormalized["image1.couleurs"])  #Voir comment le créer, et pourquoi le créer en double.
      for nameImage in L:
            for image in nameImage :
                  colors = str(image) + ".couleurs"
                  print(colors)
                  np.append(numarray,jsonMetaNormalized[colors]) 
                  print(numarray)
      
      result = np.array(jsonUserNormalized["image1.couleurs"])

      
      perceptron = Perceptron(max_iter=1000)
      perceptron.fit(numarray, result)
      
      x_predict = np.array( [jsonMetaNormalized["image1.couleurs"],jsonMetaNormalized["image2.couleurs"] ,jsonMetaNormalized["image6.couleurs"] , jsonMetaNormalized["image16.couleurs"]])

    print(dataFrame)
    print(likeFrame)
    dtc = tree.DecisionTreeClassifier()
    dtc = dtc.fit(dataFrame,likeFrame)
    
    #modelParam={}
    #modelParam['coef'] = list(perceptron.coef_)
    #modelParam['intercept']=perceptron.intercept_.tolist()

def createUser(nom, prenom, images,like):
    jsonUser=json.load(open("jsonUser.json"))
    # perceptron = Perceptron(max_iter=1000)
    # modelParam={}
    # modelParam['coef']=[images]
    # modelParam['intercept']=[like]
    # print(perceptron)

    
    doc={"nom": nom,"prenom" :prenom, "data" : {"like":[],"annee":"","format":"","taille":"","tagFav":"","model":"","couleursFav":""}}
    jsonUser.append(doc)
    jsonUserStr = json.dumps(jsonUser, indent=4)
    f = open("jsonUser.json", "w")
    f.write(jsonUserStr)
    f.close()  
#createUser("myName", "effef")
analyse(3)


## Analyse

In [86]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Get image data
jsonMetaDF = pd.read_json("metaDataJson.json")
lstTags = jsonMetaDF.loc["tags"].to_list()
lstOrientation = jsonMetaDF.loc["orientation"].to_list()

data = []
for item in jsonMetaDF:
    imageDesc = []
    imageDesc.append(jsonMetaDF.loc["dom"][item])
    imageDesc.append(jsonMetaDF.loc["tags"][item][0])
    imageDesc.append(jsonMetaDF.loc["size"][item])
    imageDesc.append(jsonMetaDF.loc["orientation"][item])
    data.append(imageDesc)

# Get user like
userDF = pd.read_json("jsonUser.json")
userIndex = 0 # Get user0
userLike = userDF.loc["data"][userIndex]["like"]
userLikeLst = []

result = []
for item in jsonMetaDF:
    if item in userLike:
        result.append("Favorite")
    else :
        result.append("NotFavorite")


# creating dataframes
dataframe = pd.DataFrame(data, columns=["dom", "tags", "size", "orientation"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# generating numerical labels
le1 = LabelEncoder()
dataframe["dom"] = le1.fit_transform(dataframe["dom"])

le2 = LabelEncoder()
dataframe["tags"] = le2.fit_transform(dataframe["tags"])

le3 = LabelEncoder()
dataframe["size"] = le3.fit_transform(dataframe["size"])

le4 = LabelEncoder()
dataframe["orientation"] = le4.fit_transform(dataframe["orientation"])

le5 = LabelEncoder()
resultframe["favorite"] = le5.fit_transform(resultframe["favorite"])

# Use of decision tree classifiers
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe.values, resultframe)

# prediction
prediction = dtc.predict(
    [
        [
            le1.transform(["Blue"])[0],
            le2.transform(["Dog"])[0],
            le3.transform(["Grande"])[0],
            le4.transform(["portrait"])[0],
        ]
    ]
)
print(le5.inverse_transform(prediction))
print(dtc.feature_importances_)

['NotFavorite']
[0.67520216 0.20215633 0.12264151 0.        ]
5


## Brouillon


image1


KeyError: 'image1'

In [ ]:
# def visualisation(parametres,nameUser='ALL'):
    
#     myJson = json.loads("metaDataJson.json")
#     myJsonNormalized = pd.json_normalize(myJson)
#     L=[]

#     jsonUser=json.loads("jsonUser.json")
#     jsonUserNormalized = pd.json_normalize(jsonUser)
#     L = jsonUserNormalized[nameUser]["image"]
#     yearData =[]
#     modelData =[]
#     for image in L:
#         if 'year' in myJsonNormalized[image]['exif']:
#             yearData.append(myJsonNormalized[image]['exif']['year'])
#         if 'mode1' in myJsonNormalized[image]:
#             modelData.append(myJsonNormalized[image]['exif']['model'])
#     max(yearData, key=yearData.count)
#     max(modelData, key=modelData.count)

# visualisation